In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate, Input

IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 3

def conv_block(x, filters, kernel_size=3, padding="same", strides=1):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    return x

# Define the input tensor
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

# Load the VGG16 model with ImageNet weights and without the fully connected layers
vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=inputs)

# Get the output tensors from the desired layers
conv1 = vgg16.get_layer("block1_conv2").output  # 64 filters
conv2 = vgg16.get_layer("block2_conv2").output  # 128 filters
conv3 = vgg16.get_layer("block3_conv3").output  # 256 filters
conv4 = vgg16.get_layer("block4_conv3").output  # 512 filters
conv5 = vgg16.get_layer("block5_conv3").output  # 512 filters

# Decoder block 1 (between conv5 and conv4)
up6 = UpSampling2D(size=(2, 2))(conv5)
concat6 = concatenate([up6, conv4], axis=-1)
conv6 = conv_block(concat6, 512)

# Decoder block 2 (between conv6 and conv3)
up7 = UpSampling2D(size=(2, 2))(conv6)
concat7 = concatenate([up7, conv3], axis=-1)
conv7 = conv_block(concat7, 256)

# Decoder block 3 (between conv7 and conv2)
up8 = UpSampling2D(size=(2, 2))(conv7)
concat8 = concatenate([up8, conv2], axis=-1)
conv8 = conv_block(concat8, 128)

# Decoder block 4 (between conv8 and conv1)
up9 = UpSampling2D(size=(2, 2))(conv8)
concat9 = concatenate([up9, conv1], axis=-1)
conv9 = conv_block(concat9, 64)

# Output layer
output = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# Create the model
model = Model(inputs=[inputs], outputs=[output])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1 (Conv2D)         │ (None, 128, 128, 64)      │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2 (Conv2D)         │ (None, 128, 128, 64)      │          36,928 │ block1_conv1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_pool (MaxPooling2D)    │ (None, 64, 64, 64)        │               0 │ block1_conv2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_conv1 (Conv2D)         │ (None, 64, 64, 128)       │          73,856 │ block1_pool[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_conv2 (Conv2D)         │ (None, 64, 64, 128)       │         147,584 │ block2_conv1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 32, 32, 128)       │               0 │ block2_conv2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block3_conv1 (Conv2D)         │ (None, 32, 32, 256)       │         295,168 │ block2_pool[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block3_conv2 (Conv2D)         │ (None, 32, 32, 256)       │         590,080 │ block3_conv1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block3_conv3 (Conv2D)         │ (None, 32, 32, 256)       │         590,080 │ block3_conv2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block3_pool (MaxPooling2D)    │ (None, 16, 16, 256)       │               0 │ block3_conv3[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block4_conv1 (Conv2D)         │ (None, 16, 16, 512)       │       1,180,160 │ block3_pool[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block4_conv2 (Conv2D)         │ (None, 16, 16, 512)       │       2,359,808 │ block4_conv1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block4_conv3 (Conv2D)         │ (None, 16, 16, 512)       │       2,359,808 │ block4_conv2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block4_pool (MaxPooling2D)    │ (None, 8, 8, 512)         │               0 │ block4_conv3[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block5_conv1 (Conv2D)         │ (None, 8, 8, 512)         │       2,359,808 │ block4_pool[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block5_conv2 (Conv2D)         │ (None, 8, 8, 512)         │       2,359,808 │ block5_conv1[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 24,891,137 (94.95 MB)

 Trainable params: 24,891,137 (94.95 MB)

 Non-trainable params: 0 (0.00 B)